In [7]:
import torch
from onnx_tf.backend import prepare
import onnx
import tensorflow as tf

In [8]:
loaded = torch.jit.load('temp/best.pt')
print(loaded)

RecursiveScriptModule(
  original_name=DualNet
  (conv1): RecursiveScriptModule(original_name=Conv2d)
  (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
  (relu): RecursiveScriptModule(original_name=ReLU)
  (layers): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(
      original_name=BasicBlock
      (conv1): RecursiveScriptModule(original_name=Conv2d)
      (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
      (relu): RecursiveScriptModule(original_name=ReLU)
      (conv2): RecursiveScriptModule(original_name=Conv2d)
      (bn2): RecursiveScriptModule(original_name=BatchNorm2d)
      (downsample): RecursiveScriptModule(original_name=Sequential)
    )
    (1): RecursiveScriptModule(
      original_name=BasicBlock
      (conv1): RecursiveScriptModule(original_name=Conv2d)
      (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
      (relu): RecursiveScriptModule(original_name=ReLU)
      (conv2): RecursiveScriptModule(original_nam

In [9]:
loaded.eval()
dummy_input = torch.randn((16, 9, 5, 5))
torch.onnx.export(loaded, dummy_input, "dualnet.onnx", example_outputs=torch.rand((16, 27)), verbose=True)

graph(%x.1 : Float(16, 9, 5, 5, strides=[225, 25, 5, 1], requires_grad=0, device=cpu),
      %pi.4.bias : Float(26, strides=[1], requires_grad=0, device=cuda:0),
      %pi.4.weight : Float(26, 800, strides=[800, 1], requires_grad=0, device=cuda:0),
      %v.4.bias : Float(32, strides=[1], requires_grad=0, device=cuda:0),
      %v.4.weight : Float(32, 75, strides=[75, 1], requires_grad=0, device=cuda:0),
      %v.6.bias : Float(1, strides=[1], requires_grad=0, device=cuda:0),
      %v.6.weight : Float(1, 32, strides=[32, 1], requires_grad=0, device=cuda:0),
      %129 : Float(32, 9, 3, 3, strides=[81, 9, 3, 1], requires_grad=0, device=cuda:0),
      %130 : Float(32, strides=[1], requires_grad=0, device=cuda:0),
      %132 : Float(32, 32, 3, 3, strides=[288, 9, 3, 1], requires_grad=0, device=cuda:0),
      %133 : Float(32, strides=[1], requires_grad=0, device=cuda:0),
      %135 : Float(32, 32, 3, 3, strides=[288, 9, 3, 1], requires_grad=0, device=cuda:0),
      %136 : Float(32, strides=

In [10]:
tf.autograph.experimental.do_not_convert
onnx_model = onnx.load('dualnet.onnx')

tf_rep = prepare(onnx_model, device='cpu')
tf_rep.export_graph('model.pb')

INFO:tensorflow:Assets written to: model.pb\assets


In [11]:
new_model = tf.saved_model.load('model.pb')
f = new_model.signatures["serving_default"]
print(f)

<ConcreteFunction signature_wrapper(x.1) at 0x25F0F0E9D30>


In [12]:
!tensorflowjs_converter --input_format=tf_saved_model --output_node_names='output_0' model.pb js_model --quantize_float16

'tensorflowjs_converter' �́A�����R�}���h�܂��͊O���R�}���h�A
����\�ȃv���O�����܂��̓o�b�` �t�@�C���Ƃ��ĔF������Ă��܂���B
